In [ ]:
# authorize google account to use bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

# authorize drive to pull model
from google.colab import drive
drive.mount('/content/drive')

# pull data for repo from bigquery
project_id = 'uic-capstone-int'
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

Authenticated
Mounted at /content/drive


In [ ]:
from tensorflow import keras
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt


#Genesis model for weight initialization
model_withTransfer = keras.models.load_model('drive/Shared drives/IDS 560 - Fall 20/model101.h5')


In [ ]:
#Unique repo id's generated since 2011 (these id's are collected from bigquery to be used in a training loop that we'll be building around the pipeline)
unique_repo_df = pd.read_csv('drive/My Drive/unique_repoid/bq-results-20200929-031543-adwz8kt104m3.csv')

In [ ]:
unique_repo_id = unique_repo_df['repoID'].tolist()

In [ ]:
import random
unique_repo_id= random.sample(unique_repo_id, 1000)

In [ ]:


for i in range(len(unique_repo_id)):

    id= unique_repo_id[i]




    # This cell is for data fetching process

    #-----------------------Pushes---------------------------------------------------

    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, count (*) number_of_pushes
    FROM
    (

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2015
    WHERE type='PushEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2016
    WHERE type='PushEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2017
    WHERE type='PushEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2018
    WHERE type='PushEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2019
    WHERE type='PushEvent' AND repo.id = {0}
    )
    GROUP BY 1,2
    ORDER BY 1;'''.format(id)
    dfPushes=client.query(sql).to_dataframe()




    #---------------------------------Forks------------------------------------------

    sql='''

    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, count (*) number_of_forks
    FROM
    (

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2015
    WHERE type='ForkEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2016
    WHERE type='ForkEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2017
    WHERE type='ForkEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2018
    WHERE type='ForkEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2019
    WHERE type='ForkEvent' AND repo.id = {0}
    )
    GROUP BY 1,2
    ORDER BY 1;
    '''.format(id)
    dfForks=client.query(sql).to_dataframe()




    #-----------------------------------------Pulls------------------------------------
    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, count (*) number_of_pulls
    FROM
    (

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2015
    WHERE type='PullRequestEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2016
    WHERE type='PullRequestEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2017
    WHERE type='PullRequestEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2018
    WHERE type='PullRequestEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2019
    WHERE type='PullRequestEvent' AND repo.id = {0}
    )
    GROUP BY 1,2
    ORDER BY 1;'''.format(id)
    dfPulls=client.query(sql).to_dataframe()



    #---------------------------------------Bookmarks-------------------------------------
    sql= '''

    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, count (DISTINCT actorID) number_of_Bookmarks
    FROM
    (
    SELECT created_at as date, repo.id as repoID, actor.id AS actorID
    FROM githubarchive.year.2015
    WHERE type='WatchEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS actorID
    FROM githubarchive.year.2016
    WHERE type='WatchEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS actorID
    FROM githubarchive.year.2017
    WHERE type='WatchEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS actorID
    FROM githubarchive.year.2018
    WHERE type='WatchEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS actorID
    FROM githubarchive.year.2019
    WHERE type='WatchEvent' AND repo.id = {0}
    )
    GROUP BY 1,2
    ORDER BY 1;'''.format(id)
    dfBoomarks = client.query(sql).to_dataframe()






    #----------------------------Issues---------------------------------------------
    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, count (*) number_of_issues
    FROM
    (

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2015
    WHERE type='IssueEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2016
    WHERE type='IssueEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2017
    WHERE type='IssueEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2018
    WHERE type='IssueEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID
    FROM githubarchive.year.2019
    WHERE type='IssueEvent' AND repo.id = {0}
    )
    GROUP BY 1,2
    ORDER BY 1;'''.format(id)
    dfIssues=client.query(sql).to_dataframe()






    #-------------------------Issuecomments-------------------------------------------

    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, COUNT(*) AS issue_comment_count, COUNT( DISTINCT a_id) as issue_author_count
    FROM 
    (SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2015
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}


    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2016
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}

    UNION ALL



    SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2017
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}

    UNION ALL


    SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2018
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}

    UNION ALL


    SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2019
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}

    )
    GROUP BY 1,2
    ORDER BY 1;'''.format(id)
    dfIssuecomment = client.query(sql).to_dataframe()



    #---------------------------------------Commitcomments------------------------------------------
    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, COUNT(*) AS commit_comment_count, COUNT( DISTINCT a_id) AS commit_author_count
    FROM 
    (SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2015
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}


    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2016
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}

    UNION ALL



    SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2017
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}

    UNION ALL


    SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2018
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}

    UNION ALL


    SELECT created_at as date, repo.id as repoID, actor.id AS a_id
    FROM githubarchive.year.2019
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}

    )
    GROUP BY 1,2
    ORDER BY 1;'''.format(id)
    dfCommitcomment = client.query(sql).to_dataframe()






    merge1 = pd.merge(dfPushes,dfForks,how='outer', on='year_month')
    merge2 = pd.merge(merge1, dfPulls, how='outer', on='year_month')
    merge3 = pd.merge(merge2, dfBoomarks, how='outer', on='year_month')
    merge4 = pd.merge(merge3, dfIssuecomment, how='outer', on='year_month')
    merge5 = pd.merge(merge4,dfCommitcomment,how='outer',on='year_month')
    merge6 = pd.merge(merge5, dfIssues, how='outer',on='year_month')
    merge6.drop(['repoID_x','repoID_y'], axis=1, inplace=True)
    merge6.drop('repoID', axis=1, inplace=True)
    merge6.sort_values(by='year_month',ascending=True,inplace=True)
    merge6





    merge6['year_month']=pd.to_datetime(merge6['year_month'])
    merge6.rename({'year_month': 'datetime'}, axis=1, inplace=True)
    merge6.index=merge6['datetime']
    merge6.drop('datetime',axis=1,inplace=True)
    merge6




    #This step is for padding the dataframe to maintain the consistency of all the time series we'll be fetching in the future
    #This step also deals with the problem of missing dates(if no event happens at all) in the time series
    from datetime import datetime
    now=datetime.now()
    today = now.strftime("%Y-%m-%d")
    today

    idx = pd.date_range('2016-01-01', today)
    merge6.index = pd.DatetimeIndex(merge6.index)

    merge6 = merge6.reindex(idx, fill_value=0)


    #aggregating the weekly data and creating weekly time steps
    WeeklyTS = pd.DataFrame()
    WeeklyTS = merge6.resample('W').sum()


    #Summing all the data to get an idea of the frequency of different events
    col_sum=WeeklyTS.sum(axis=0)
    col_sum.sort_values(ascending=False, inplace=True)







    #dimensionality reduction just for the visualization

    # from sklearn.decomposition import PCA
    # pca = PCA(n_components=2).fit(WeeklyTS)
    # pca_2d = pca.transform(WeeklyTS)





    #clustering just to see any patterns and to create a dummy target variable
    from sklearn.cluster import KMeans
    Kmeansmodel = KMeans(n_clusters=2)
    Kmeansmodel.fit(WeeklyTS)
    y_kmeans = Kmeansmodel.predict(WeeklyTS)




    #Just adding a dummy target variable for testing
    WeeklyTS['status']=Kmeansmodel.labels_
    WeeklyTS['status'].value_counts()



    #seperating date feature into 'year','month', and 'day' features
    WeeklyTS.insert(0, 'year', WeeklyTS.index.year)
    WeeklyTS.insert(1, 'month', WeeklyTS.index.month)
    WeeklyTS.insert(2, 'day', WeeklyTS.index.day)
    WeeklyTS.reset_index(level=0, inplace=True)



    WeeklyTS.drop('index', axis=1, inplace=True)
    WeeklyTS



    # yet to be performed - Normalization of the above dataset



    # multivariate data preparation

    from numpy import array
    from numpy import hstack

    # split a multivariate sequence into samples
    def split_sequences(sequences, n_steps):
      X, y = list(), list()
      for i in range(len(sequences)):
        # end of the pattern
        end_ix = i + n_steps
        # check if surpaas the dataset
        if end_ix > len(sequences):
          break
        # input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
      return array(X), array(y)
    

    # define input sequence
    t=[]

    # transforming each column to an array
    for i in range(0,len(WeeklyTS.columns)):
      t.append(WeeklyTS.iloc[:,i].to_numpy())

    # convert array to [rows, columns] structure
    for i in range(0,len(WeeklyTS.columns)):
      t[i]=t[i].reshape((len(t[i]),1))



    # horizontally stack columns
    dataset=t[0]
    for i in range(1,len(WeeklyTS.columns)):
      dataset = hstack((dataset,t[i]))

    # Window Size
    n_steps = 5

    # convert into input/output
    X, y = split_sequences(dataset, n_steps)
    n_features = X.shape[2]
    




    #Vanilla model
    def build_LSTM(n_steps, hidden_units):
      model = Sequential()
      model.add(LSTM(hidden_units, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
      model.add(LSTM(hidden_units, activation='relu'))
      model.add(Dense(n_features))
      model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
      return model





    def FitForecast(X_train, y_train, n_steps, hidden_units, epochs, trained_model):

          '''
          Fits a model and returns the model and history
          Args:
              X_train: input dataset for training
              y_train: output dataset for training
              n_steps: window size
              hidden_units: int/list specifying the number 
                            of hidden units
              epochs: int that defines the number of 
                      training phases through the
                      training dataset
              trained_model: already trained keras sequential 
                            model
              
          Returns:
              model: keras sequential model
              history: training loss history and training accuracy history
              
          '''

          model = build_LSTM(n_steps,hidden_units)

          #initializing with the previous model weights
          model.set_weights(weights = trained_model.get_weights())        

          history = model.fit(x=X_train, y=y_train, epochs=epochs, verbose=0,shuffle=False)

          return model, history



    model_withTransfer, hist = FitForecast(X, y, 5, 20, 400, trained_model=model_withTransfer)
    print("Training completed-------------------->repoID:%d" %id)




In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.plot(hist.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train accuracy'], loc='upper right')
plt.show()

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.plot(hist.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss'], loc='upper right')
plt.show()